In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/SC_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9547507704,SC,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14
1,9777677704,SC,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14
2,8416237102,SC,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,8936977105,SC,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,9301987000,SC,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9547507704,SC,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14
1,9777677704,SC,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14
2,8416237102,SC,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,8936977105,SC,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,9301987000,SC,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
772,2269407205,SC,"1921 Falls Valley Drive, RALEIGH, SC",35.898383,-78.607021,NaN
4852,7167417003,SC,"203 E Main St 2nd floor, SPARTANBURG, SC",42.421830,-94.986230,NaN
6151,2264257100,SC,"PO BOX 4540, FLORENCE, SC",34.195433,-79.762563,NaN
7481,2754087100,SC,"3545 Highway 17 Bypass, MYRTLE BEACH, SC",28.059534,-81.734188,NaN
9882,4317487702,SC,"8354 State Rd, Philadelphia, SC",40.035567,-75.015204,NaN
10409,9793947702,SC,"17 Lockwood Drive, Suite 400, DANIEL ISLAND, SC",32.779924,-79.952364,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('45')] #SC
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/6 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

6


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14283/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
772,2269407205,SC,"1921 Falls Valley Drive, RALEIGH, SC",35.898383,-78.607021,[]
4852,7167417003,SC,"203 E Main St 2nd floor, SPARTANBURG, SC",42.421830,-94.986230,[]
6151,2264257100,SC,"PO BOX 4540, FLORENCE, SC",34.195433,-79.762563,[450410010002015]
7481,2754087100,SC,"3545 Highway 17 Bypass, MYRTLE BEACH, SC",28.059534,-81.734188,[]
9882,4317487702,SC,"8354 State Rd, Philadelphia, SC",40.035567,-75.015204,[]
10409,9793947702,SC,"17 Lockwood Drive, Suite 400, DANIEL ISLAND, SC",32.779924,-79.952364,[450190005002024]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14283/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
772,2269407205,SC,"1921 Falls Valley Drive, RALEIGH, SC",35.898383,-78.607021,NaN
4852,7167417003,SC,"203 E Main St 2nd floor, SPARTANBURG, SC",42.421830,-94.986230,NaN
6151,2264257100,SC,"PO BOX 4540, FLORENCE, SC",34.195433,-79.762563,450410010002015
7481,2754087100,SC,"3545 Highway 17 Bypass, MYRTLE BEACH, SC",28.059534,-81.734188,NaN
9882,4317487702,SC,"8354 State Rd, Philadelphia, SC",40.035567,-75.015204,NaN
10409,9793947702,SC,"17 Lockwood Drive, Suite 400, DANIEL ISLAND, SC",32.779924,-79.952364,450190005002024


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
772,2269407205,SC,"1921 Falls Valley Drive, RALEIGH, SC",35.898383,-78.607021,NaN
4852,7167417003,SC,"203 E Main St 2nd floor, SPARTANBURG, SC",42.421830,-94.986230,NaN
7481,2754087100,SC,"3545 Highway 17 Bypass, MYRTLE BEACH, SC",28.059534,-81.734188,NaN
9882,4317487702,SC,"8354 State Rd, Philadelphia, SC",40.035567,-75.015204,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
772,2269407205,NaN
4852,7167417003,NaN
6151,2264257100,450410010002015
7481,2754087100,NaN
9882,4317487702,NaN
10409,9793947702,450190005002024


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/SC_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,778677,9547507704,SC,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14
1,778678,9777677704,SC,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14
2,778679,8416237102,SC,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14
3,778680,8936977105,SC,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14
4,778681,9301987000,SC,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,778677,9547507704,SC,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14,NaN
1,778678,9777677704,SC,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14,NaN
2,778679,8416237102,SC,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14,NaN
3,778680,8936977105,SC,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14,NaN
4,778681,9301987000,SC,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14,NaN
...,...,...,...,...,...,...,...,...
10736,789413,8896048610,SC,"450 Park St, Inman, SC",35.063540,-82.087490,4.508302e+14,NaN
10737,789414,9147798304,SC,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,4.500302e+14,NaN
10738,789415,9543158509,SC,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,4.501502e+14,NaN
10739,789416,9671458706,SC,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,4.501502e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,778677,9547507704,SC,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,4.508500e+14,NaN,450850019032026.0
1,778678,9777677704,SC,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,4.501900e+14,NaN,450190031132025.0
2,778679,8416237102,SC,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,4.508500e+14,NaN,450850013002008.0
3,778680,8936977105,SC,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,4.508302e+14,NaN,450830231014043.0
4,778681,9301987000,SC,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,4.504500e+14,NaN,450450016003021.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9547507704,SC,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,450850019032026.0
1,9777677704,SC,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,450190031132025.0
2,8416237102,SC,"104 N MAIN ST, SUMTER, SC",33.921860,-80.341545,450850013002008.0
3,8936977105,SC,"337 SPARTANGREEN BLVD, DUNCAN, SC",34.918210,-82.095220,450830231014043.0
4,9301987000,SC,"105 N Spring St, GREENVILLE, SC",34.897892,-82.363490,450450016003021.0
...,...,...,...,...,...,...
10736,8896048610,SC,"450 Park St, Inman, SC",35.063540,-82.087490,450830229022004.0
10737,9147798304,SC,"126 Vivion Dr, Aiken, SC",33.538596,-81.745072,450030212023011.0
10738,9543158509,SC,"224 Seven Farms Dr Ste 101, Daniel Island, SC",32.860386,-79.907030,450150204075006.0
10739,9671458706,SC,"245 Seven Farms Dr Ste 110, Daniel Island, SC",32.859726,-79.908760,450150204075032.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
772,2269407205,SC,"1921 Falls Valley Drive, RALEIGH, SC",35.898383,-78.607021,NaN
4852,7167417003,SC,"203 E Main St 2nd floor, SPARTANBURG, SC",42.421830,-94.986230,NaN
7481,2754087100,SC,"3545 Highway 17 Bypass, MYRTLE BEACH, SC",28.059534,-81.734188,NaN
9882,4317487702,SC,"8354 State Rd, Philadelphia, SC",40.035567,-75.015204,NaN


Convert to File

In [19]:
fips_SC = fips_merge_drop

In [20]:
fips_SC.to_csv("../../../data/state_data/geo/geo_fips/23/SC_fips_scraped.csv") 